# 📄 이미지 PDF OCR + 레이아웃 추출 파이프라인

목적: 전체 파일 처리 파이프라인에서 이미지 기반 PDF를 자동 판별하여 OCR 대상으로 분류하고, 페이지 레이아웃 정보를 바탕으로 텍스트/이미지/표 데이터를 추출·저장하며 전체 텍스트 파일을 생성합니다.

핵심 기능:
- 이미지 PDF 자동 판별(텍스트 적음 → OCR 대상)
- 페이지 단위 OCR 실행 및 실시간 진행 로그
- 페이지 이미지 저장, 표 추출(pdfplumber 기반) 및 저장
- 전체 텍스트 파일 및 메타데이터 JSON 저장

In [2]:
# ✅ 의존성 점검 (빠른 확인 전용: 설치는 별도)
import importlib, sys


def check_lib(name):
    try:
        importlib.import_module(name)
        print(f'✔ {name} 사용 가능', flush=True)
        return True
    except Exception as e:
        print(f'✖ {name} 불가: {e}', flush=True)
        return False


needed = ['pdf2image', 'pytesseract', 'pdfplumber', 'PIL']
print('🔎 라이브러리 상태 확인:', flush=True)
status = {n: check_lib(n) for n in needed}
missing = [n for n, ok in status.items() if not ok]
if missing:
    print(f'⚠ 설치 필요: {", ".join(missing)}', flush=True)
else:
    print('✅ 모든 필수 라이브러리 확인됨', flush=True)

🔎 라이브러리 상태 확인:
✔ pdf2image 사용 가능
✔ pdf2image 사용 가능
✔ pytesseract 사용 가능
✖ pdfplumber 불가: No module named 'pdfminer.pdftypes'
✔ PIL 사용 가능
⚠ 설치 필요: pdfplumber
✔ pytesseract 사용 가능
✖ pdfplumber 불가: No module named 'pdfminer.pdftypes'
✔ PIL 사용 가능
⚠ 설치 필요: pdfplumber


In [3]:
# ⚙️ 경로/설정
from pathlib import Path
from dataclasses import dataclass, field
from typing import Optional, Dict, Any
import time, json

BASE_DIR = Path('/home/admin/wkms-aws/jupyter_notebook')
INPUT_PDF = BASE_DIR / 'data/input_docs/test.pdf'  # 필요시 변경
OUTPUT_DIR = BASE_DIR / 'data/opensource_output/extracted_texts'

@dataclass
class OCRConfig:
    ocr_enabled: bool = True
    ocr_language: str = 'eng'  # 예: 'kor' 또는 'kor+eng'
    ocr_dpi: int = 220

@dataclass
class Paths:
    output_root: Path = OUTPUT_DIR
    page_texts: Path = field(default_factory=lambda: OUTPUT_DIR)
    page_images: Path = field(default_factory=lambda: OUTPUT_DIR / 'page_images')
    page_tables: Path = field(default_factory=lambda: OUTPUT_DIR / 'page_tables')

paths = Paths()
for d in [paths.output_root, paths.page_images, paths.page_tables]:
    d.mkdir(parents=True, exist_ok=True)

ocr_cfg = OCRConfig()
print('📂 출력 경로 설정 완료:', flush=True)
print(f' - 페이지 텍스트: {paths.page_texts}', flush=True)
print(f' - 페이지 이미지: {paths.page_images}', flush=True)
print(f' - 표 출력: {paths.page_tables}', flush=True)

📂 출력 경로 설정 완료:
 - 페이지 텍스트: /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts
 - 페이지 이미지: /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts/page_images
 - 표 출력: /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts/page_tables
 - 페이지 텍스트: /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts
 - 페이지 이미지: /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts/page_images
 - 표 출력: /home/admin/wkms-aws/jupyter_notebook/data/opensource_output/extracted_texts/page_tables


In [4]:
# 🔧 유틸리티
def write_text(path: Path, content: str):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w', encoding='utf-8') as f:
        f.write(content)

def write_json(path: Path, data: Dict[str, Any]):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def is_image_pdf(pdf_path: Path, sample_pages: int = 3) -> bool:
    """간단 기준: pdfplumber로 앞부분 텍스트 추출 시 매우 적으면 이미지 기반으로 판단"""
    try:
        import pdfplumber  # type: ignore
        total_txt = ''
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages[:sample_pages], 1):
                try:
                    total_txt += (page.extract_text() or '')
                except Exception:
                    pass
        return len((total_txt or '').strip()) < 100
    except Exception:
        # 라이브러리 없거나 실패 시 보수적으로 OCR 대상으로 간주
        return True

In [5]:
# 🔤 OCR + 페이지별 저장 (텍스트/이미지)

def ocr_pdf_to_text(pdf_path: Path, paths: Paths, cfg: OCRConfig) -> Dict[str, Any]:
    import time
    from pathlib import Path as _P
    try:
        from pdf2image import convert_from_path  # type: ignore
        import pytesseract  # type: ignore
        from PIL import Image  # type: ignore
    except Exception as e:
        raise RuntimeError(f'OCR 실행에 필요한 라이브러리가 없습니다: {e}')

    start = time.time()
    print(f'🔄 PDF→이미지 변환 중 (DPI={cfg.ocr_dpi})...', flush=True)
    images = convert_from_path(str(pdf_path), dpi=cfg.ocr_dpi)
    print(f'   ✅ 변환 완료: {len(images)}페이지', flush=True)

    full_text_parts = []
    page_text_files = []
    page_image_files = []
    page_meta = []

    for idx, img in enumerate(images, 1):
        p_start = time.time()
        print(f'   🔍 페이지 {idx}/{len(images)} OCR...', flush=True)
        txt = pytesseract.image_to_string(img, lang=cfg.ocr_language)
        txt = txt.strip()
        page_txt_path = paths.page_texts / f'{pdf_path.stem}_page_{idx:03d}.txt'
        write_text(page_txt_path, txt)
        page_text_files.append(str(page_txt_path))

        page_img_path = paths.page_images / f'{pdf_path.stem}_page_{idx:03d}.png'
        try:
            img.save(page_img_path)
            page_image_files.append(str(page_img_path))
        except Exception:
            pass

        full_text_parts.append(f'\n\n=== 페이지 {idx} ===\n' + txt)
        page_meta.append({
            'page': idx,
            'text_length': len(txt),
            'processing_time': round(time.time() - p_start, 3),
            'text_file': str(page_txt_path),
            'image_file': str(page_img_path) if page_image_files else None
        })

    full_text = '\n'.join(full_text_parts).strip()
    full_txt_path = paths.output_root / f'{pdf_path.stem}_ocr_full.txt'
    write_text(full_txt_path, full_text)

    meta = {
        'method': 'ocr',
        'pages': len(images),
        'ocr_language': cfg.ocr_language,
        'ocr_dpi': cfg.ocr_dpi,
        'total_time': round(time.time() - start, 3),
        'page_details': page_meta,
        'output_file': str(full_txt_path),
        'page_text_files': page_text_files,
        'page_image_files': page_image_files
    }
    return {'text': full_text, 'metadata': meta}

In [8]:
# 📊 표 추출(pdfplumber) — 벡터 기반 PDF에서 유효. 이미지 PDF에선 보통 없음

def extract_tables_with_pdfplumber(pdf_path: Path, paths: Paths, max_pages: Optional[int] = None) -> Dict[str, Any]:
    try:
        import pdfplumber  # type: ignore
    except Exception as e:
        print(f'⚠ pdfplumber 불가: {e}', flush=True)
        return {'tables': [], 'files': []}

    saved = []
    tables_summary = []
    with pdfplumber.open(str(pdf_path)) as pdf:
        pages = pdf.pages if max_pages is None else pdf.pages[:max_pages]
        for idx, page in enumerate(pages, 1):
            try:
                tbls = page.extract_tables() or []
            except Exception:
                tbls = []
            if not tbls:
                continue
            # 각 테이블을 간단한 텍스트로 저장
            for t_i, table in enumerate(tbls, 1):
                lines = [' | '.join([(c or '').strip() for c in row]) for row in table if row]
                content = (f'=== 페이지 {idx} 테이블 {t_i} ===\n' + '\n'.join(lines)).strip()
                out = paths.page_tables / f'{pdf_path.stem}_p{idx:03d}_t{t_i:02d}.txt'
                write_text(out, content)
                saved.append(str(out))
                tables_summary.append({'page': idx, 'table_index': t_i, 'rows': len(table)})
    return {'tables': tables_summary, 'files': saved}

In [9]:
# 🧠 오케스트레이터: 이미지 PDF 판별 → OCR → 표 추출 → 결과 저장

def process_pdf(pdf_path: Path, paths: Paths, cfg: OCRConfig) -> Dict[str, Any]:
    print(f'📥 입력 파일: {pdf_path}', flush=True)
    if not pdf_path.exists():
        raise FileNotFoundError(f'파일이 없습니다: {pdf_path}')
    img_pdf = is_image_pdf(pdf_path)
    print(f'🧪 이미지 PDF 판별: {"예" if img_pdf else "아니오"}', flush=True)

    # 1) 텍스트/전체 파일 추출
    if img_pdf or cfg.ocr_enabled:
        print('🔤 OCR 경로로 처리합니다.', flush=True)
        res = ocr_pdf_to_text(pdf_path, paths, cfg)
    else:
        # 텍스트 기반 PDF 간단 추출 (보조)
        try:
            import pdfplumber  # type: ignore
            print('📝 텍스트 기반 추출(pdfplumber.extract_text) 시도', flush=True)
            texts = []
            with pdfplumber.open(str(pdf_path)) as pdf:
                for idx, page in enumerate(pdf.pages, 1):
                    t = page.extract_text() or ''
                    write_text(paths.page_texts / f'{pdf_path.stem}_page_{idx:03d}.txt', (t or '').strip())
                    texts.append(f'\n\n=== 페이지 {idx} ===\n' + (t or '').strip())
            full = '\n'.join(texts).strip()
            out = paths.output_root / f'{pdf_path.stem}_full.txt'
            write_text(out, full)
            res = {'text': full, 'metadata': {'method': 'text', 'output_file': str(out)}}
        except Exception as e:
            print(f'⚠ 텍스트 추출 실패, OCR로 대체: {e}', flush=True)
            res = ocr_pdf_to_text(pdf_path, paths, cfg)

    # 2) 표 추출 (가능한 경우)
    print('📊 표 추출 시도(pdfplumber)', flush=True)
    table_info = extract_tables_with_pdfplumber(pdf_path, paths)

    # 3) 메타데이터 합성
    meta = res.get('metadata', {})
    meta.update({
        'tables': table_info.get('tables', []),
        'table_files': table_info.get('files', [])
    })
    doc_meta_path = paths.output_root / f'{pdf_path.stem}_metadata.json'
    write_json(doc_meta_path, meta)

    print('✅ 처리 완료', flush=True)
    print(f' - 전체 텍스트 파일: {meta.get("output_file")}', flush=True)
    print(f' - 표 파일 수: {len(meta.get("table_files", []))}', flush=True)
    print(f' - 페이지 텍스트 파일 수: {len(meta.get("page_text_files", []))}', flush=True)
    print(f' - 페이지 이미지 파일 수: {len(meta.get("page_image_files", []))}', flush=True)
    return {'text': res.get('text', ''), 'metadata': meta}

In [10]:
# ▶️ 실행 러너: test.pdf 대상으로 데모 실행
pdf_path = INPUT_PDF
print('🚀 이미지 PDF OCR + 레이아웃 추출 데모 시작', flush=True)
print(f'입력: {pdf_path}', flush=True)
result = process_pdf(pdf_path, paths, ocr_cfg)

# 상위 500자 미리보기
preview = (result.get('text') or '')[:500]
print('\n🔎 전체 텍스트 미리보기(상위 500자):', flush=True)
print(preview if preview else '(미리보기 없음)', flush=True)

🚀 이미지 PDF OCR + 레이아웃 추출 데모 시작
입력: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
📥 입력 파일: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
🧪 이미지 PDF 판별: 예
🔤 OCR 경로로 처리합니다.
🔄 PDF→이미지 변환 중 (DPI=220)...
입력: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
📥 입력 파일: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
🧪 이미지 PDF 판별: 예
🔤 OCR 경로로 처리합니다.
🔄 PDF→이미지 변환 중 (DPI=220)...
   ✅ 변환 완료: 27페이지
   🔍 페이지 1/27 OCR...
   ✅ 변환 완료: 27페이지
   🔍 페이지 1/27 OCR...
   🔍 페이지 2/27 OCR...
   🔍 페이지 2/27 OCR...
   🔍 페이지 3/27 OCR...
   🔍 페이지 3/27 OCR...
   🔍 페이지 4/27 OCR...
   🔍 페이지 4/27 OCR...
   🔍 페이지 5/27 OCR...
   🔍 페이지 5/27 OCR...
   🔍 페이지 6/27 OCR...
   🔍 페이지 6/27 OCR...
   🔍 페이지 7/27 OCR...
   🔍 페이지 7/27 OCR...
   🔍 페이지 8/27 OCR...
   🔍 페이지 8/27 OCR...
   🔍 페이지 9/27 OCR...
   🔍 페이지 9/27 OCR...
   🔍 페이지 10/27 OCR...
   🔍 페이지 10/27 OCR...
   🔍 페이지 11/27 OCR...
   🔍 페이지 11/27 OCR...
   🔍 페이지 12/27 OCR...
   🔍 페이지 12/27 OCR...
   🔍 페이지 13/27 OCR...
  

## 참고
- Tesseract OCR이 시스템에 설치되어 있어야 합니다 (예: Ubuntu: `sudo apt-get install -y tesseract-ocr`).
- 한국어 OCR은 `kor` 데이터가 필요합니다 (예: `sudo apt-get install -y tesseract-ocr-kor` 또는 학습데이터 설치).
- 표 추출은 벡터 기반 PDF에서만 동작할 수 있습니다. 이미지 PDF의 표는 OCR문자열 기반으로 후처리해야 하며, 본 노트북은 기본적인 텍스트 추출을 우선 수행합니다.
- 출력물:
  - 전체 텍스트 파일: `<파일명>_ocr_full.txt` 또는 `<파일명>_full.txt`
  - 페이지별 텍스트 파일: `<파일명>_page_###.txt`
  - 페이지 이미지: `page_images/` 폴더 내 PNG
  - 표 텍스트: `page_tables/` 폴더 내 텍스트 파일
  - 문서 메타데이터 JSON: `<파일명>_metadata.json`